# Transfer

In [1]:
%load_ext jupyter_black
%matplotlib inline

In [2]:
import os
from pathlib import Path
from typing import Iterable

import xarray as xr

import mmmpy


root = Path(os.path.abspath(__name__)).parents[1]
root

PosixPath('/workspaces/MMM-Py')

In [4]:
netcdf, binary, grib = sorted(root.glob("data/*.zip"))
netcdf, binary, grib

(PosixPath('/workspaces/MMM-Py/data/mosaic3d_tile6_20130531-231500.netcdf.zip'),
 PosixPath('/workspaces/MMM-Py/data/mrms_binary_data.zip'),
 PosixPath('/workspaces/MMM-Py/data/mrms_grib_data.zip'))

In [7]:
with mmmpy.io.unzip2(netcdf) as f:
    mrms = mmmpy.io.read_mrms(f, "netcdf")
mrms

<xarray.Dataset>
Dimensions:       (Ht: 31, Lat: 2001, Lon: 2001)
Dimensions without coordinates: Ht, Lat, Lon
Data variables:
    mrefl_mosaic  (Ht, Lat, Lon) int16 dask.array<chunksize=(31, 2001, 2001), meta=np.ndarray>
    Height        (Ht) float32 dask.array<chunksize=(31,), meta=np.ndarray>
Attributes:
    TypeName:        mrefl_mosaic
    DataType:        LatLonHeightGrid
    Time:            1370042100
    FractionalTime:  0.0
    MissingData:     -999.0
    RangeFolded:     -1000.0
    Latitude:        40.0
    Longitude:       -110.0
    Height:          500.0
    LatGridSpacing:  0.01
    LonGridSpacing:  0.01
    attributes:

In [4]:
mrms= mmmpy.read_mrms(netcdf)
mrms

<xarray.Dataset>
Dimensions:       (Ht: 31, Lat: 2001, Lon: 2001)
Dimensions without coordinates: Ht, Lat, Lon
Data variables:
    mrefl_mosaic  (Ht, Lat, Lon) int16 dask.array<chunksize=(31, 2001, 2001), meta=np.ndarray>
    Height        (Ht) float32 dask.array<chunksize=(31,), meta=np.ndarray>
Attributes:
    TypeName:        mrefl_mosaic
    DataType:        LatLonHeightGrid
    Time:            1370042100
    FractionalTime:  0.0
    MissingData:     -999.0
    RangeFolded:     -1000.0
    Latitude:        40.0
    Longitude:       -110.0
    Height:          500.0
    LatGridSpacing:  0.01
    LonGridSpacing:  0.01
    attributes:

In [12]:
@mmmpy.unzip("/tmp/mmmpy/")
def mfread_netcdf(files: Iterable[Path], filetype: str):
    ds = xr.open_mfdataset(
        files,
        engine="netcdf4",
        chunks={},
        backend_kwargs=dict(
            mask_and_scale=True,
            decode_times=True,
            concat_characters=True,
            decode_coords=True,
            use_cftime=None,
            decode_timedelta=None,
            group=None,
            mode="r",
            format="NETCDF4",
            clobber=True,
            diskless=False,
            persist=False,
            lock=None,
            autoclose=False,
        ),
    )
    return ds


mfread_netcdf(netcdf)

<xarray.Dataset>
Dimensions:       (Ht: 31, Lat: 2001, Lon: 2001)
Dimensions without coordinates: Ht, Lat, Lon
Data variables:
    mrefl_mosaic  (Ht, Lat, Lon) int16 dask.array<chunksize=(31, 2001, 2001), meta=np.ndarray>
    Height        (Ht) float32 dask.array<chunksize=(31,), meta=np.ndarray>
Attributes:
    TypeName:        mrefl_mosaic
    DataType:        LatLonHeightGrid
    Time:            1370042100
    FractionalTime:  0.0
    MissingData:     -999.0
    RangeFolded:     -1000.0
    Latitude:        40.0
    Longitude:       -110.0
    Height:          500.0
    LatGridSpacing:  0.01
    LonGridSpacing:  0.01
    attributes:

In [4]:
mrms = mmmpy.read_mrms(netcdf)
mrms


<xarray.Dataset>
Dimensions:       (Ht: 31, Lat: 2001, Lon: 2001)
Dimensions without coordinates: Ht, Lat, Lon
Data variables:
    mrefl_mosaic  (Ht, Lat, Lon) int16 dask.array<chunksize=(31, 2001, 2001), meta=np.ndarray>
    Height        (Ht) float32 dask.array<chunksize=(31,), meta=np.ndarray>
Attributes:
    TypeName:        mrefl_mosaic
    DataType:        LatLonHeightGrid
    Time:            1370042100
    FractionalTime:  0.0
    MissingData:     -999.0
    RangeFolded:     -1000.0
    Latitude:        40.0
    Longitude:       -110.0
    Height:          500.0
    LatGridSpacing:  0.01
    LonGridSpacing:  0.01
    attributes:

In [1]:


@mmmpy.unzip("/tmp/mmmpy/")
def mfread_cfgrib(files: Iterable[Path], filetype:str):
    ds = xr.open_mfdataset(
        tuple(files)[:2],
        engine="cfgrib",
        concat_dim=["heightAboveSea"],
        combine="nested",
        chunks={},
        # backend_kwargs=None
        backend_kwargs=dict(
            mask_and_scale=True,
            decode_times=True,
            concat_characters=True,
            decode_coords=True,
            # drop_variables=None,
            use_cftime=None,
            decode_timedelta=None,
            lock=None,
            indexpath="{path}.{short_hash}.idx",
            filter_by_keys={},
            read_keys=[],
            encode_cf=("parameter", "time", "geography", "vertical"),
            squeeze=True,
            time_dims=("time", "step"),
        ),
    )
    return ds


mrms = mfread_cfgrib(grib)
mrms

NameError: name 'mmmpy' is not defined